# Circuit non-determinism

Zero Knowledge Proofs allow a prover to prove to the verfier that they know a solution to some problem without revealing any additional information. For example, you could prove that you know a pre-image of a hash without revealing the actual preimage. Modern systems are complex and expressive enough to prove complex programs (these are often called zkVMs). Most used proof systems create succinct proofs. A few kilobytes is all that needs to be sent to the verfier to prove that you've won in tetris (you can emulate the program).

However, to make all of this possible, we first need to represent the computation in such a way that can be later compiled into a proof by the proving system. This is what circuits are for. Circuits consist of gates, where each gate enforces an algebraic relation on the values in the wires (for simplicity just think about those as vectors).

For example, let's say we have a circuit of size 4 that enforces the fibonacci sequence with two wires:

+ `w_1 = [1, 2, 5, 13]`
+ `w_2 = [1, 3, 8, 21]`

The values in the wires are called witnesses

We want to enforce several relations on this circuit:
1. Everywhere but the last gate `w_1_next - w_1 - w_2 == 0` (you can't look at previous values usually, only the one in the next gate)
2. Everywhere but the last gate `w_2_next - w_1_next - w_2 ==0`
3. At the first gate only `w_1 - 1 == 0`
4. At the first gate only `w_2 - 1 == 0`

As you can see, we need the logic to differ at various indices. To enable or disable relations at various gates we can use selectors. These are additional vectors that are fixed and can't be changed once we decide on the circuit. In this case we can use:
+ `lagrange_first = [1, 0, 0, 0]`
+ `lagrange_last = [0, 0, 0, 1]`

The system of relations that we are checking on each gate is then:
1. `lagrange_first * (w_1 - 1) == 0`
2. `lagrange_first * (w_2 - 1) == 0`
3. `(1 - lagrange_last) * (w_1_next - w_1 - w_2 == 0)`
4. `(1 - lagrange_last) * (w_2_next - w_1_next - w_2 == 0)`


By the way, all computation has be on the elements of a given Finite Field. Different systems use different finite fields. For example, Succint Non-Interactive Arguments of Knowledge (SNArKs) often use larger scalar fields of Elliptic Curves, while Scalable Transparent ARguments of Knowledge (STArKs) can use smaller more convenient fields, but this comes with other trade-offs.


## PLONK gate model (high level)

Obviously, most of the time we don't want to create a completely new circuit system for each type of program we want to convert into a circuit. That would require a lot of effort and time. So there are several systems, where you only have to configure the selectors appropriately for the computation you want to prove. One of the most basic and widespread ones is Plonk (https://eprint.iacr.org/2019/953.pdf).

To constrain the arithmetic computation in each gate it uses 5 selectors:
1. `q_m` (multiplication)
2. `q_l` (left selector)
3. `q_r` (right selector)
4. `q_o` (output selector)
5. `q_c` (constant selector)

It also uses 3 wires:
1. `w_l` (left)
2. `w_r` (right)
3. `w_o` (output)

The relation is then as follows:

`q_m * w_l * w_r + q_l * w_l + q_r * w_r + q_o * w_o + q_c == 0`

You might have noticed that we have not used any values from the next index. That's because Plonk also has additional witnesses and relations that allow it to connect any witness indices, so you could enforce, for example, `w_l[10]==w_r[15]`. However, that is a complex argument and out of the current scope, so we'll ignore that particular relation for now and just emulate it.

The Plonk gate is very expressive. You can create various types of gates:
1. Addition(`q_m=0, q_l=1, q_r=1,q_o=-1, q_c=0`): `w_l + w_r - w_o == 0`
2. Multiplication(`q_m=1, q_l=0, q_r=0, q_o=-1, q_c=0`): `w_l * w_r - w_o == 0`
3. Fixed witness(`q_m=0, q_l=-1, q_r=0, q_o=0, q_c=<SomeConstant>`): `w_l - <SomeConstant> == 0`. This enforce the value of the witness
4. Boolean gate(`q_m=1, q_l=-1, q_r=0, q_o=0, q_c=0`): `w_l * w_r - w_l == 0` and we enforce `w_l==w_r` throught the permutation argument, so it becomes `w_l * w_l - w_l == 0`. This ensures the value of the witness is 0 or 1
5. Xor gate (`q_m=2, q_l=-1, q_r=-1, q_o=1, q_c=0`): `2 * w_l * w_r - w_l - w_r + w_o == 0`. If both left and right witness have been constrained to be boolean before, this enforces 1-bit Xor

Let's create a simple plonk circuit

In [240]:
# Use the alt_bn128 scalar field
Fr_modulus = 21888242871839275222246405745257275088548364400416034343698204186575808495617

# Let's define a simple field element class
class Fr:
    def __init__(self, value):
        if isinstance(value, Fr):
            value = value.value
        self.value = value % Fr_modulus
    def __add__(self, other):
        return Fr(self.value + other.value)
    def __mul__(self, other):
        return Fr(self.value * other.value)
    def __neg__(self):
        return Fr(Fr_modulus - self.value)
    def __sub__(self, other):
        return self + (-other)
    def __eq__(self, other):
        return self.value == other.value
    def __str__(self):
        return f"Fr({self.value})"
    def invert(self):
        return self.pow(Fr_modulus - 2)
    def __div__(self, other):
        return self * other.invert()
    def __truediv__(self, other):
        return self * other.invert()
    def __neg__(self):
        return Fr(Fr_modulus - self.value)
    def pow(self, power):
        power = power % (Fr_modulus - 1)
        return Fr(pow(self.value, power, Fr_modulus))
    def __repr__(self):
        return self.__str__()

# The class for building the circuit
class PlonkCircuitBuilder:
    def __init__(self):
        """Initialize the circuit builder"""
        self.w_l = []
        self.w_r = []
        self.w_o = []
        self.q_m = []
        self.q_l = []
        self.q_r = []
        self.q_o = []
        self.q_c = []
        # We are going to use the same variable in different gates. This allows us to enforce the permutation argument
        # This way we can connect w_l[10]==w_r[15]
        self.variables = []
        # Let's create a convenient variable for 0
        self.zero_index = self.add_variable(Fr(0))
        # And enforce that it's 0
        self.create_fixed_witness_gate(self.zero_index, Fr(0))
    def add_variable(self, variable_value):
        """Add a variable to the circuit"""
        # We remember the value of the variable
        self.variables.append(variable_value)
        # And return the index of the variable. The value doesn't affect the structure of the circuit
        return len(self.variables) - 1

    def get_circuit_size(self):
        """Get the size of the circuit"""
        return len(self.q_m)

    def replace_variables(self, new_variables):
        """Replace the variables with new ones"""
        # We are going to use this method later during the challenge
        assert len(new_variables) == len(self.variables)
        self.variables = new_variables

    def _format_fr_short(self, fr_value):
        """Format the field element to a short string"""
        if fr_value is None:
            return "None"
        v = fr_value.value if isinstance(fr_value, Fr) else int(fr_value)
        if v == 0:
            return "0"
        neg = (Fr_modulus - v) % Fr_modulus
        pos_str = str(v)
        neg_str = "-" + str(neg)
        return neg_str if len(neg_str) < len(pos_str) else pos_str

    def create_fixed_witness_gate(self, variable_index, witness_value):
        """Create a fixed witness gate"""
        self.q_m.append(Fr(0))
        self.q_l.append(Fr(-1))
        self.q_r.append(Fr(0))
        self.q_o.append(Fr(0))
        self.q_c.append(Fr(witness_value))
        self.w_l.append(variable_index)
        self.w_r.append(0)
        self.w_o.append(0)

    def create_boolean_gate(self, variable_index):
        """Create a boolean gate"""
        self.q_m.append(Fr(1))
        self.q_l.append(Fr(-1))
        self.q_r.append(Fr(0))
        self.q_o.append(Fr(0))
        self.q_c.append(Fr(0))
        self.w_l.append(variable_index)
        self.w_r.append(variable_index)
        self.w_o.append(0)
    def create_xor_gate(self, left_index, right_index, output_index):
        """Create a xor gate"""
        self.q_m.append(Fr(2))
        self.q_l.append(Fr(-1))
        self.q_r.append(Fr(-1))
        self.q_o.append(Fr(1))
        self.q_c.append(Fr(0))
        self.w_l.append(left_index)
        self.w_r.append(right_index)
        self.w_o.append(output_index)
    def check_circuit(self):
        """Check that the witnesses in the variables satisfy the circuit"""
        circuit_size = len(self.q_m)
        for i in range(circuit_size):
            # We check that each gate is satisfied
            if (
                self.q_m[i] * self.variables[self.w_l[i]] * self.variables[self.w_r[i]]
                + self.q_l[i] * self.variables[self.w_l[i]]
                + self.q_r[i] * self.variables[self.w_r[i]]
                + self.q_o[i] * self.variables[self.w_o[i]]
                + self.q_c[i]
            ) != Fr(0):
                return False
        return True
    def get_variables(self):
        """Get the variables"""
        return self.variables
    def print_gates(self, show_values=False):
        """Print the gates"""
        def add_term(coeff_str, body_str, is_first):
            if coeff_str == "0":
                return "", is_first
            is_negative = coeff_str.startswith("-")
            magnitude = coeff_str[1:] if is_negative else coeff_str
            if is_first:
                prefix = "-" if is_negative else ""
            else:
                prefix = " - " if is_negative else " + "
            if body_str:
                term = body_str if magnitude == "1" else f"{magnitude}*{body_str}"
            else:
                term = magnitude
            return prefix + term, False
        gate_count = len(self.q_m)
        for gate_index in range(gate_count):
            left_witness_index = self.w_l[gate_index]
            right_witness_index = self.w_r[gate_index]
            output_witness_index = self.w_o[gate_index]
            qm_fr = self.q_m[gate_index]
            ql_fr = self.q_l[gate_index]
            qr_fr = self.q_r[gate_index]
            qo_fr = self.q_o[gate_index]
            qc_fr = self.q_c[gate_index]
            qm_s = self._format_fr_short(qm_fr)
            ql_s = self._format_fr_short(ql_fr)
            qr_s = self._format_fr_short(qr_fr)
            qo_s = self._format_fr_short(qo_fr)
            qc_s = self._format_fr_short(qc_fr)
            expr = ""
            first = True
            term, first = add_term(qm_s, f"w[{left_witness_index}]*w[{right_witness_index}]", first)
            expr += term
            term, first = add_term(ql_s, f"w[{left_witness_index}]", first)
            expr += term
            term, first = add_term(qr_s, f"w[{right_witness_index}]", first)
            expr += term
            term, first = add_term(qo_s, f"w[{output_witness_index}]", first)
            expr += term
            term, first = add_term(qc_s, "", first)
            expr += term
            if expr == "":
                expr = "0"
            line = f"gate {gate_index}: {expr} == 0"
            if (
                qm_fr == Fr(2)
                and ql_fr == Fr(-1)
                and qr_fr == Fr(-1)
                and qo_fr == Fr(1)
                and qc_fr == Fr(0)
            ):
                line += "  # XOR gate"
            elif (
                qm_fr == Fr(1)
                and ql_fr == Fr(-1)
                and qr_fr == Fr(0)
                and qo_fr == Fr(0)
                and qc_fr == Fr(0)
            ):
                line += "  # BOOLEAN gate"
            if show_values:
                left_fr = (
                    self.variables[left_witness_index]
                    if left_witness_index < len(self.variables)
                    else None
                )
                right_fr = (
                    self.variables[right_witness_index]
                    if right_witness_index < len(self.variables)
                    else None
                )
                output_fr = (
                    self.variables[output_witness_index]
                    if output_witness_index < len(self.variables)
                    else None
                )
                line += (
                    " | a="
                    + self._format_fr_short(left_fr)
                    + ", b="
                    + self._format_fr_short(right_fr)
                    + ", c="
                    + self._format_fr_short(output_fr)
                )
            print(line)
    # Ignore the rest of the code in this class for now
    def create_2bit_xor_gate(self, left_index, right_index, output_index):
        """Create a 2-bit xor gate"""
        assert left_index < len(self.variables)
        assert right_index < len(self.variables)
        assert output_index < len(self.variables)
        assert left_index >= 0
        assert right_index >= 0
        assert output_index >= 0
        value_left = self.variables[left_index].value
        value_right = self.variables[right_index].value
        value_output = self.variables[output_index].value
        value_left_low = value_left & 1
        value_left_high = value_left >> 1
        value_right_low = value_right & 1
        value_right_high = value_right >> 1
        value_output_low = value_output & 1
        value_output_high = value_output >> 1
        left_low_index = self.add_variable(Fr(value_left_low))
        left_high_index = self.add_variable(Fr(value_left_high))
        right_low_index = self.add_variable(Fr(value_right_low))
        right_high_index = self.add_variable(Fr(value_right_high))
        output_low_index = self.add_variable(Fr(value_output_low))
        output_high_index = self.add_variable(Fr(value_output_high))
        self.create_xor_gate(left_low_index, right_low_index, output_low_index)
        self.create_xor_gate(left_high_index, right_high_index, output_high_index)
        self.create_boolean_gate(output_low_index)
        self.create_boolean_gate(output_high_index)
        self.create_boolean_gate(left_low_index)
        self.create_boolean_gate(left_high_index)
        self.create_boolean_gate(right_low_index)
        self.create_boolean_gate(right_high_index)

    def create_generic_gate(self, left_index, right_index, output_index, q_m, q_l, q_r, q_o, q_c):
        """Create a generic gate"""
        self.q_m.append(q_m)
        self.q_l.append(q_l)
        self.q_r.append(q_r)
        self.q_o.append(q_o)
        self.q_c.append(q_c)
        self.w_l.append(left_index)
        self.w_r.append(right_index)
        self.w_o.append(output_index)

    def create_64_bit_xor_gate(self, left_index, right_index, output_index):
        """Create a 64-bit xor gate"""
        assert left_index < len(self.variables)
        assert right_index < len(self.variables)
        assert output_index < len(self.variables)
        assert left_index >= 0
        assert right_index >= 0
        assert output_index >= 0
        value_left = self.variables[left_index].value
        value_right = self.variables[right_index].value
        value_output = self.variables[output_index].value
        assert value_left.bit_length() <= 64
        assert value_right.bit_length() <= 64
        assert value_output.bit_length() <= 64
        current_left_accumulator = value_left
        current_right_accumulator = value_right
        current_output_accumulator = value_output
        current_left_index = left_index
        current_right_index = right_index
        current_output_index = output_index
        for i in range(31):
            current_left_bits = current_left_accumulator & 3
            current_right_bits = current_right_accumulator & 3
            current_output_bits = current_output_accumulator & 3
            new_left_accumulator = current_left_accumulator >> 2
            new_right_accumulator = current_right_accumulator >> 2
            new_output_accumulator = current_output_accumulator >> 2
            new_left_accumulator_index = self.add_variable(Fr(new_left_accumulator))
            new_right_accumulator_index = self.add_variable(Fr(new_right_accumulator))
            new_output_accumulator_index = self.add_variable(Fr(new_output_accumulator))
            low_bits_left_index = self.add_variable(Fr(current_left_bits))
            low_bits_right_index = self.add_variable(Fr(current_right_bits))
            low_bits_output_index = self.add_variable(Fr(current_output_bits))
            self.create_generic_gate(
                current_left_index,
                new_left_accumulator_index,
                low_bits_left_index,
                Fr(0),
                Fr(1),
                Fr(-4),
                Fr(-1),
                Fr(0),
            )
            self.create_generic_gate(
                current_right_index,
                new_right_accumulator_index,
                low_bits_right_index,
                Fr(0),
                Fr(1),
                Fr(-4),
                Fr(-1),
                Fr(0),
            )
            self.create_generic_gate(
                current_output_index,
                new_output_accumulator_index,
                low_bits_output_index,
                Fr(0),
                Fr(1),
                Fr(-4),
                Fr(-1),
                Fr(0),
            )
            self.create_2bit_xor_gate(
                low_bits_left_index, low_bits_right_index, low_bits_output_index
            )
            current_left_index = new_left_accumulator_index
            current_right_index = new_right_accumulator_index
            current_output_index = new_output_accumulator_index
            current_left_accumulator = new_left_accumulator
            current_right_accumulator = new_right_accumulator
            current_output_accumulator = new_output_accumulator
        self.create_2bit_xor_gate(
            current_left_index, current_right_index, current_output_index
        )
    



In [241]:
# Build and inspect a 1-bit XOR circuit
# First, we create a circuit builder
c = PlonkCircuitBuilder()
# Then, we add the input and output variables
left = c.add_variable(Fr(1))
right = c.add_variable(Fr(0))
output = c.add_variable(Fr(1))
# We create boolean gates to enforce that the values are boolean
c.create_boolean_gate(left)
c.create_boolean_gate(right)
c.create_boolean_gate(output)
# Then, we create the xor gate
c.create_xor_gate(left, right, output)
# We can print the circuit size and the gates
print(f"Circuit size (rows): {c.get_circuit_size()}")
c.print_gates(show_values=True)

# The assignmend of variables should satisfy the circuit in this case
print("Passing circuit check:", c.check_circuit())

# If we change the right variable, the circuit will be unsatisfied
c.variables[right] = Fr(1)
print("Failed circuit check:", c.check_circuit())

# We can also check the values of the variables
print(c.variables)


Circuit size (rows): 5
gate 0: -w[0] == 0 | a=0, b=0, c=0
gate 1: w[1]*w[1] - w[1] == 0  # BOOLEAN gate | a=1, b=1, c=0
gate 2: w[2]*w[2] - w[2] == 0  # BOOLEAN gate | a=0, b=0, c=0
gate 3: w[3]*w[3] - w[3] == 0  # BOOLEAN gate | a=1, b=1, c=0
gate 4: 2*w[1]*w[2] - w[1] - w[2] + w[3] == 0  # XOR gate | a=1, b=0, c=1
Passing circuit check: True
Failed circuit check: False
[Fr(0), Fr(1), Fr(1), Fr(1)]


## Onto non-determinism

Writing circuits is hard and developers often make mistakes. Some mistakes don't allow proving a correct statement (for example, you would not be able to prove that 1^1==0). These are called Completeness bugs. Some mistakes allow proving an incorrect statement. These are called Soundness bugs (and are usually worse).

Coming back to the xor circuit. Imagine the developer wanted to Xor a value with itself, but instead used a different one:




In [242]:
# Build and inspect a 1-bit XOR circuit
# First, we create a circuit builder
c = PlonkCircuitBuilder()
value = 1
result = value^value
# Then, we add the input and output variables (the developer forgot that add_variable doesn't constraint to the field)
left = c.add_variable(Fr(value))
right = c.add_variable(Fr(value))
output = c.add_variable(Fr(result))
# We create boolean gates to enforce that the values are boolean
c.create_boolean_gate(left)
c.create_boolean_gate(output)
# Then, we create the xor gate
c.create_xor_gate(left, right, output)

assert c.check_circuit()

The problem now becomes that we can prove to the verifier that `left ^ left == 1`, because the circuit is underconstrained. For this method it is easy to compute the values manually, but let's try automation instead. We will use and SMT solver:

## Installing and using Z3 in Python

Install the Python bindings:

```bash
pip install z3-solver
```

Basic usage pattern:
- Create symbolic variables and constraints
- Ask the solver for `sat`
- Extract a model



### Installing the native Z3 solver (not just the Python wrapper)

The `z3-solver` Python package usually ships with a prebuilt native Z3 library for common platforms. If your platform lacks a prebuilt wheel or you get import errors, install the native solver via your OS package manager:

- Linux (Ubuntu/Debian):
```bash
sudo apt update && sudo apt install -y z3
```
- macOS (Homebrew):
```bash
brew install z3
```
- Windows (Chocolatey):
```powershell
choco install z3
```
- Alternative: download binaries from the official releases (`https://github.com/Z3Prover/z3/releases`) and add the library to your PATH/LD_LIBRARY_PATH.

Tip: If you’re in a virtual environment, you still install Z3 system-wide; the Python package will find the system library if a wheel isn’t available.


In [243]:
# Verify Z3 Python package and native library availability
try:
    import z3
    from z3 import Solver, Int, sat
    print("Imported z3 from:", z3.__file__)
    # Version info (tuple or string depending on z3 build)
    try:
        from z3 import get_version_string
        print("Z3 version:", get_version_string())
    except Exception:
        try:
            from z3 import get_version
            print("Z3 version:", get_version())
        except Exception:
            pass
    # Sanity check a trivial constraint
    s = Solver()
    x = Int('x')
    s.add(x > 0)
    assert s.check() == sat
    print("Z3 sanity check: OK (sat)")
    # Try to touch the underlying native library
    try:
        from z3.z3core import lib
        # Call a trivial native function via wrapper
        print("Native Z3 library is loaded: OK")
    except Exception as e:
        print("Warning: Could not confirm native library via z3core.lib:", e)
except Exception as e:
    print("Z3 import or runtime failed:", e)
    print("If you installed only the Python wrapper but lack a native lib, try:")
    print(" - Ubuntu/Debian: sudo apt install z3")
    print(" - macOS (Homebrew): brew install z3")
    print(" - Windows (Chocolatey): choco install z3")
    print("Or download from https://github.com/Z3Prover/z3/releases and add to PATH/LD_LIBRARY_PATH")


Imported z3 from: /home/rumata888/miniforge3/lib/python3.12/site-packages/z3/__init__.py
Z3 version: 4.15.3
Z3 sanity check: OK (sat)


In [244]:
from z3 import Solver, Int, BitVec, BV2Int, sat
# Using z3 is very simple
# We create a solver    
s = Solver()
# We can then create variables
x = Int('x')
y = Int('y')
# We can add constraints
constraint_1=x+y==9
constraint_2=x*y==20
s.add(constraint_1)
s.add(constraint_2)
# We can check if the constraints are satisfiable
print(s.check())
# If they are, we can get a model
model = s.model()
print(model)
# We can also get the values of the variables
print(model[x], model[y])




sat
[x = 4, y = 5]
4 5


To check the non-determinism of a circuit we can create two sets of formulas. Then we equate the inputs and enforce the outputs to be different:

In [245]:
# Let's print the circuit:
c.print_gates(show_values=False)

gate 0: -w[0] == 0
gate 1: w[1]*w[1] - w[1] == 0  # BOOLEAN gate
gate 2: w[3]*w[3] - w[3] == 0  # BOOLEAN gate
gate 3: 2*w[1]*w[2] - w[1] - w[2] + w[3] == 0  # XOR gate


In [246]:
# Create new solver
s = Solver()
# Create variables
witness_1 = [Int(f'witness_1_{i}') for i in range(len(c.variables))]
witness_2 = [Int(f'witness_2_{i}') for i in range(len(c.variables))]
# Ensure that we enforce all witnesses to be in the field
for witness_element in witness_1:
    s.add(witness_element>=0)
    s.add(witness_element<Fr_modulus)
for witness_element in witness_2:
    s.add(witness_element>=0)
    s.add(witness_element<Fr_modulus)

def create_formulas_for_circuit(witness):
    formulas = []
    # Gate 0: left == 0
    formulas.append(witness[0]==0)
    # Gate 1: boolean gate
    formulas.append((witness[1]*witness[1]-witness[1])%Fr_modulus==0)
    # Gate 2: boolean gate
    formulas.append((witness[2]*witness[2]-witness[2])%Fr_modulus==0)
    # Gate 3: boolean gate
    formulas.append((witness[3]*witness[3]-witness[3])%Fr_modulus==0)
    # Gate 4: xor gate
    formulas.append((2*witness[1]*witness[2]-witness[1]-witness[2]+witness[3])%Fr_modulus==0)
    return formulas

# Create formulas for each copy of the circuit
for formula in create_formulas_for_circuit(witness_1):
    s.add(formula)
for formula in create_formulas_for_circuit(witness_2):
    s.add(formula)

# Connect inputs (only variable with index 1 is considered as input)
s.add(witness_1[1]==witness_2[1])
# Enforce that the outputs are different (we treat variable with index 3 as output)
s.add(witness_1[3]!=witness_2[3])

print(s.check())

#print(s.model())

witness_1_model = [s.model()[witness_1[i]] for i in range(len(witness_1))]
witness_2_model = [s.model()[witness_2[i]] for i in range(len(witness_2))]

print ("First circuit:")
print(f"{witness_1_model[1]}^{witness_1_model[2]}=={witness_1_model[3]}")
print ("Second circuit:")
print(f"{witness_2_model[1]}^{witness_2_model[2]}=={witness_2_model[3]}")


        

sat
First circuit:
0^1==1
Second circuit:
0^0==0


We have successfully detected an issue in the circuit and found a problematic witness! Now for the fun part. The following circuit is incorrect (I have deliberately underconstrained some gates). You purpose is find such inputs that the `left^right!=output`. You can play with the local server emulator first. Then submit the witness to the server and get the flag!

In [247]:
def vulnerable_circuit():
    circuit = PlonkCircuitBuilder()
    input_0 = circuit.add_variable(Fr(0))
    input_1 = circuit.add_variable(Fr(0))
    output = circuit.add_variable(Fr(0))
    circuit.create_64_bit_xor_gate(input_0, input_1, output)
    return circuit , input_0, input_1, output

circuit, input_0, input_1, output = vulnerable_circuit()

def send_witness_to_emulated_server(witness):
    circuit,_,_,_=vulnerable_circuit()
    circuit.replace_variables(witness)
    if circuit.check_circuit() and witness[1].value^witness[2].value!=witness[3].value:
        return "Contragulations! You got the flag! CRYPOTRAINING{NOT_A_REAL_FLAG}"
    else:
        return "Failed to verify the circuit or the witness is not malicious"

def find_malicious_witness():
    # Replace the code in this function with your own code
    c,_,_,_=vulnerable_circuit()
    # initial variables
    variables=c.get_variables()
    return variables

print (send_witness_to_emulated_server(find_malicious_witness()))

Failed to verify the circuit or the witness is not malicious


## Connecting to the server and submitting a witness

The server expects a single line with all circuit variables as space-separated integers. It will:
- Replace its internal witness with yours
- Verify the circuit constraints
- If the inputs do not XOR to the output, return the flag

Adjust `HOST` and `PORT` as needed.


In [ ]:
import socket

def send_witness(host, port, witness):
    line = " ".join(str(x) for x in witness) + "\n"
    with socket.create_connection((host, port), timeout=10.0) as sock:
        try:
            sock.settimeout(2.0)
            _ = sock.recv(4096)  # read banner/prompt
        except Exception:
            pass
        sock.settimeout(10.0)
        sock.sendall(line.encode())
        chunks = []
        try:
            while True:
                chunk = sock.recv(4096)
                if not chunk:
                    break
                chunks.append(chunk)
        except socket.timeout:
            pass
        return (b"".join(chunks)).decode(errors="ignore")

# Example usage:
HOST = "cryptoraining.zone"; PORT = 1356
resp = send_witness(HOST, PORT, find_malicious_witness())
print(resp)


Inputs xor to the output. This does not break the circuit. Try again.

